In [73]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

In [74]:
chromedriver = 'E:\Build\Project\programmers\crawling\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(15)
driver.get('https://www.agoda.com/ko-kr/search?guid=e81a9725-13c8-4a40-8485-b7088addeb00&asq=u2qcKLxwzRU5NDuxJ0kOF3T91go8JoYYMxAgy8FkBH1BN0lGAtYH25sdXoy34qb93onxmKnxpuaVVXrUZ1yJUXxn9jc2K1y2tFWc0J4g099DFVZT4hIRosKR1%2Fo3Ev2xXiyX8%2F8HJ3jSDjfHoaOyVZ1HYEzEOqyvdox%2BwS6yxHdu2KyFu6KsBzdkHUtq2RTYmarYnqAY%2BRBDXLfH7%2FDaoQ%3D%3D&city=14690&tick=637484992472&isdym=true&searchterm=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&txtuuid=e81a9725-13c8-4a40-8485-b7088addeb00&locale=ko-kr&ckuid=e7d5fd20-00a8-4b0d-acce-f1e937e9c1af&prid=0&currency=KRW&correlationId=43ccc528-67f8-4c96-90d5-9ee13a7c9db6&pageTypeId=103&realLanguageId=9&languageId=9&origin=KR&cid=-1&userId=e7d5fd20-00a8-4b0d-acce-f1e937e9c1af&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=26&currencyCode=KRW&htmlLanguage=ko-kr&cultureInfoName=ko-kr&machineName=hk-crweb-2012&trafficGroupId=4&sessionId=gtqjqoiceuqstupaaivoflga&trafficSubGroupId=4&aid=130243&useFullPageLogin=true&cttp=4&isRealUser=true&checkIn=2021-05-20&checkOut=2021-05-21&rooms=1&adults=2&children=0&priceCur=KRW&los=1&textToSearch=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&travellerType=1&familyMode=off&productType=-1')

In [130]:
def page_full_scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    element = driver.find_element_by_tag_name('body')
    while True:
        for _ in range(15):
            element.send_keys(Keys.SPACE)
            time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("loading complete")
    
def isAgodaReviewFind(driver):
    try:
        review_tabs = driver.find_elements_by_css_selector('.Review-tab')
    except Exception as error:
        return False
    for review_tab in review_tabs:
        if 'agoda' in review_tab.get_attribute("innerHTML").lower():
            driver.execute_script("arguments[0].click();", review_tab)
            return True
    return False
def isLanguageKorean(driver):
    try:
        reviews_filters = driver.find_elements_by_css_selector('.select-dropdown-wrapper')
    except Exception as error:
        return False
    
    for reviews_filter in reviews_filters:
        if '작성 언어' in reviews_filter.get_attribute("innerHTML"):
            driver.execute_script("arguments[0].click();", reviews_filter)
            reviews_lans = reviews_filter.find_elements_by_css_selector('.select-dropdown-list__item')
            for lan in reviews_lans:
                if '한국어' in lan.get_attribute("innerHTML"):
                    driver.execute_script("arguments[0].click();", lan)
                    return True
    return False
            
def hotel_list(driver):
    global hotel_data
    global review_data
    global hotel_idx
    
    hotels = driver.find_elements_by_css_selector('.PropertyCardItem')
    hotel_len = len(hotels)
    cnt = 0
    while True:
        try:
            hotel = hotels[cnt].find_element_by_tag_name('h3')
            driver.execute_script("arguments[0].click();", hotel)
        except Exception as error:
            continue
            
        driver.switch_to.window(driver.window_handles[-1])
        
        time.sleep(5)
        
        
        hotel_name = driver.find_element_by_css_selector('.HeaderCerebrum__Name').get_attribute("innerHTML")
        try:
            hotel_address = driver.find_element_by_css_selector('.HeaderCerebrum__Address').get_attribute("innerHTML")
        except Exception as error:
            hotel_address = None
        try:
            hotel_rate = driver.find_element_by_css_selector('.Review__ReviewFormattedScore').get_attribute("innerHTML")
        except Exception as error:
            hotel_rate = None
            
        hotel = [hotel_idx, hotel_name, hotel_address, hotel_rate]
        hotel_data.append(hotel)
        
        isAgodaReview = isAgodaReviewFind(driver)
        isKoreanReview = isLanguageKorean(driver)
        
        if isAgodaReview and isKoreanReview:
            detail_review_crawling(driver)
        print(hotel_name, ' complete (', cnt+1, '/', hotel_len, ')')
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        hotel_idx+=1
        cnt += 1
        if cnt >= hotel_len:
            break  
    return hotel_data, review_data

def detail_review_crawling(driver):
    global review_data
    global hotel_idx
    cnt = 0
    while True:
        try:
            detail_reviews = driver.find_elements_by_css_selector('.Review-comment')
        except Exception as error:
            break
        for review in detail_reviews:
            score = review.find_element_by_css_selector('.Review-comment-leftScore').get_attribute("innerHTML")
            title = review.find_element_by_css_selector('.Review-comment-bodyTitle').get_attribute("innerHTML")
            text = review.find_element_by_css_selector('.Review-comment-bodyText').get_attribute("innerHTML")
            date = review.find_element_by_css_selector('.Review-statusBar-date').get_attribute("innerHTML")
            review_d = [hotel_idx, title, text, score, date[5:]]
            review_data.append(review_d)
            
        try:
            arrow = driver.find_elements_by_css_selector('.Review-paginator-arrow')
        except Exception as error:
            break
            
        inactive_arrow = driver.find_elements_by_css_selector('.Review-paginator-arrow--inactive')
        if len(arrow) < 2:
            break
        if inactive_arrow and cnt != 0:
            break
        cnt += 1
        a = arrow[1].find_element_by_css_selector('.ficon')
        driver.execute_script("arguments[0].click();", a)  
def save_csv_file(hotel_data, review_data, num):
    hotel_file_name = './data/hotel_info_data_' + str(num) + '.csv'
    review_file_name = './data/review_info_data_' + str(num) + '.csv'

    hotel_df = pd.DataFrame(hotel_data)
    review_df = pd.DataFrame(review_data)

    hotel_df.to_csv(hotel_file_name, encoding='utf-8-sig')
    review_df.to_csv(review_file_name, encoding='utf-8-sig')

In [131]:
driver.implicitly_wait(time_to_wait=10)
hotel_data = []
review_data = []
hotel_idx = 0
c = 1
while True:
    page_full_scroll(driver)
    
    print(c, 'page start -----------------')
    hotel_list(driver)
    
    save_csv_file(hotel_data, review_data, c)
    print(c, 'page complete --------------')
    c += 1
    nextBtn = driver.find_element_by_id('paginationNext')
    if nextBtn:
        nextBtn.send_keys(Keys.RETURN)
        time.sleep(3)
    else:
        break
    

loading complete
1 page start -----------------
명동 루프탑 호스텔 (Myeongdong Rooftop Hostel)  complete ( 1 / 90 )
호텔 컬리넌 개포 (Hotel Cullinan Gaepo)  complete ( 2 / 90 )
호텔 스카이파크 센트럴 명동점 (Hotel Skypark Central Myeongdong)  complete ( 3 / 90 )
케이팝 호텔 서울역 (K Pop Hotel Seoul Station)  complete ( 4 / 90 )
코지박스 게스트하우스 (Cozybox Guesthouse)  complete ( 5 / 90 )
토요코인 서울 영등포 (Toyoko Inn Seoul Yeongdeungpo)  complete ( 6 / 90 )
호텔 스카이파크 명동 3호점 (Hotel Skypark Myeongdong III)  complete ( 7 / 90 )
J 레지던스 (J Residence)  complete ( 8 / 90 )
버틀러리 올스위츠 (BUTLERLEE ALLSUITES)  complete ( 9 / 90 )
킴스테이9 (Kimstay9)  complete ( 10 / 90 )
호텔 나포레 (Hotel Nafore)  complete ( 11 / 90 )
호텔 스테이 인 서울역 (Hotel Stay Inn Seoul Station)  complete ( 12 / 90 )
보리 호텔 강남 (Boree Hotel Gangnam)  complete ( 13 / 90 )
프린세스 호텔 (Princess Hotel)  complete ( 14 / 90 )
버틀러리 폴백 (BUTLERLEE PAULBAEK)  complete ( 15 / 90 )
호텔 베뉴지 (Hotel Venue G)  complete ( 16 / 90 )
인터시티 서울 호텔 (Intercity Seoul Hotel)  complete ( 17 / 90 )
보르조미 부티크 서울 호텔 (Borjo

선샤인 호텔 (Sunshine Hotel)  complete ( 46 / 90 )
신촌 에버8 서비스 레지던스 (Shinchon Ever8 Serviced Residence)  complete ( 47 / 90 )
퍼시픽 호텔 명동 (Pacific Hotel)  complete ( 48 / 90 )
뉴서울 호텔 (New Seoul Hotel)  complete ( 49 / 90 )
남산 힐 호텔 (Namsan Hill Hotel)  complete ( 50 / 90 )
하모니 호텔 (Harmony Hotel)  complete ( 51 / 90 )
한옥 24 게스트하우스 경복궁 (Hanok 24 guesthouse Gyeongbokgung)  complete ( 52 / 90 )
호텔 메이플레이스 서울 동대문 (Hotel Mayplace Seoul Dongdaemun)  complete ( 53 / 90 )
모노하우스 홍대 2호점 (MONO HOUSE HONGDAE 2)  complete ( 54 / 90 )
베스트웨스턴 프리미어 서울 가든 호텔 (Best Western Premier Seoul Garden Hotel)  complete ( 55 / 90 )
서울숲 스테이 (Seoul Forest Stay)  complete ( 56 / 90 )
프리마 호텔 (Prima Hotel)  complete ( 57 / 90 )
호텔 쿠레타케소 인사동 (Hotel Kuretakeso Insadong)  complete ( 58 / 90 )
호텔썬비 (Sunbee Hotel Insadong Seoul)  complete ( 59 / 90 )
제이 미니 호텔 종로 인사동 (J mini Hotel Jongno Insadong)  complete ( 60 / 90 )
서울 상주호텔 (The Sangju Hotel Seoul)  complete ( 61 / 90 )
위고인 호스텔 (Wegoinn Hostel)  complete ( 62 / 90 )
베스트인 부띠크 명동 (Ve

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".HeaderCerebrum__Name"}
  (Session info: chrome=88.0.4324.146)


In [128]:

driver.close()
driver.switch_to.window(driver.window_handles[0])